In [1]:
# モデルとトークナイザのロード

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "llm-jp/llm-jp-3.1-1.8b-instruct4"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)  #--- use_fast=True ---
# tokenizer.pad_toekn = tokenizer.eos_token           # --- guitar plus ---

bnb_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    # trust_remote_code=True
)

/home/dev/miniconda3/envs/lora_ja_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# （チューニング前の）ベースモデルの出力をJSONで保存

import json
import csv

# with open("../data/guitar_eval_gold.jsonl", "r", encoding="utf-8") as f:
# with open("../data/guitar_eval_question_only.jsonl", "r", encoding="utf-8") as f:
with open("../data/koji_eval_gold.jsonl", "r", encoding="utf-8") as f:
    qa_list = [json.loads(line) for line in f]

results = []

for i, qa in enumerate(qa_list):
    prompt = f"{qa['instruction']}\n{qa['input']}\n答え:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    inputs.pop("token_type_ids", None)
    output_ids = base_model.generate(**inputs,
                                # max_new_tokens=512,
                                max_new_tokens=256,
                                 do_sample=False      # --- ここはllmjp_lora_tuning.ipynbではなく、infer_lora.pyより ---
                                 )
    
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    output_text = output_text[len(prompt):].strip()

    results.append({
        "instruction": qa["instruction"],
        "input": qa["input"],
        "output": output_text
    })

# # 1_配列JSONで保存
# with open("../data/qa_op_before_lora.json", "w", encoding="utf-8") as f:
#     json.dump(results, f, ensure_ascii=False, indent=2)


# 2_JSONL形式で保存
with open("../data/koji_before_rag.jsonl", "w", encoding="utf-8") as f:
    for obj in results:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")


# # 3_CSV形式で保存
# with open("../data/qa_op_before_lora.csv", "w", encoding="utf-8", newline="") as f:
#     writer = csv.DictWriter(f, fieldnames=["instruction", "input", "output"])
#     writer.writeheader()   # ヘッダ行を出力
#     writer.writerows(results)  # 各行を書き込み


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o